## Proposing an effective Backtesting strategy using Bayesian Modeling 

### By: Serena Liu, Lin Ye and Akshat Mittal 

### Introduction

Backtesting is a key component of effective trading system development. It is accomplished by reconstructing, with historical data, trades that would have occurred in the past using rules defined by a given strategy. The result offers statistics to gauge the effectiveness of the strategy. 




In this project, our team proposes a novel way to generate a backtesting strategy, where we make use of Bayesian Modeling to predict quarterly returns and industry groups and building our strategy based on them. Bayesian modelling is particularly helpful in the application of finance where data set is smaller and higher noise. As an extension to industry wide practice of frequentist models,  our approach incorporates probablistic interpretation to return predictions. In addition, we extend the probabilistic intepretation to clustering industries as well. We constructed industry neutral backtests with GICs industry code and our own version of industries from GMM model.   


Global Industry Classification Standard (GICS) is a framework which dilineates companies into 11 following sectors:

1. Communication services
2. Consumer discretionary
3. Consumer staples
4. Energy
5. Financials
6. Health care
7. Industrials
8. Information technology
9. Materials
10. Real estate
11. Utilities
    
(The framework further classifies companies into 24 industry groups, 68 industries, and 157 subindustries, but we limit our focus to the sector level classification)

Two things stand out for this classification:
- This classfication of companies is based not on any financial factors, but on the product or service they provide
- This is a hard classification i.e. at a certain time, companies usually have a single GICS code associated with them

In this work, we try to come up with a soft classification of companies into sectors using their values of their financial fundamental indicators using a Bayesian Gaussian Mixture Model where each component of the mixture would.

Using the posterior associations of each component for each data point, we supplant the indicator GICS in our two predictive models: Bayesian Linear Regression and Bayesian Neural Network and do a comparative analysis of our backtest for both types of models - the predictive models having GIC indicators with the predictive models having soft classifications

### Data

We use the company fundamentals data for x companies over y years at a z frequency along with the close price (which is eventually used by converting them to return values). This data was downloaded from [SimFin](https://simfin.com/), which makes fundamental financial data available for free.

Our final list of fundamentals, which will eventually serve as predictors of the return, are:



Take a look at the correlation among our features: